In [1]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv("creditcard.csv")

# Avaliação inicial do Dataset

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Primeiro podemos notar que contamos com um conjunto de 30 colunas totalmente preenchidas (sem nulos). O Dataset contém um conjunto de variáveis de V1 a V28 utilizadas para prever se a transação no cartão de crédito é fraudulenta ou não, e então classificá-la em 1 se for fraudulenta, senão 0.

In [9]:
data['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

Outro ponto que pode ser notado é o evidente Desbalanceamento deste Dataset, o que pode causar um grande viés no aprendizado de máquina.

# Aplicação do Perceptron

In [5]:
# Definindo a classe Perceptron
class Perceptron:
    def __init__(self, input_size):
        self.weights = np.random.rand(input_size)  # Inicialização dos pesos com valores aleatórios
        self.bias = np.random.rand()               # Inicializa o bias com um valor aleatório
    
    # Método para fazer previsões
    def predict(self, inputs):
        net_input = np.dot(inputs, self.weights) + self.bias  # Calcula o input líquido (soma ponderada + bias)
        activation = self.activation_function(net_input)      # Aplica a função de ativação
        return activation
    
    # Função de ativação: função degrau
    def activation_function(self, x):
        return 1 if x >= 0 else 0

    # Método de treinamento do Perceptron
    def train(self, inputs, labels, learning_rate, epochs):
        for epoch in range(epochs):  # Itera por cada época
            for i in range(len(inputs)):  # Itera por cada exemplo no conjunto de treinamento
                prediction = self.predict(inputs[i])   # Faz uma previsão
                error = labels[i] - prediction         # Calcula o erro
                # Atualiza os pesos e o bias com base no erro
                self.weights += learning_rate * error * inputs[i]
                self.bias += learning_rate * error


In [10]:
# Separando as features e a variável alvo
X_sample = data.drop('Class', axis=1).values
y_sample = data['Class'].values

# Dividindo os dados em conjuntos de treinamento e teste
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)


In [12]:
# Instanciando o modelo Perceptron
perceptron_model = Perceptron(input_size=X_train_sample.shape[1])
# Definindo a taxa de aprendizado e o número de épocas
learning_rate = 0.01
epochs = 20
# Treinando o modelo Perceptron
perceptron_model.train(X_train_sample, y_train_sample, learning_rate, epochs)

# Avaliando o desempenho do modelo no conjunto de treinamento
train_predictions = [perceptron_model.predict(x) for x in X_train_sample]
# Avaliando o desempenho do modelo no conjunto de teste
test_predictions = [perceptron_model.predict(x) for x in X_test_sample]
# Calculando a acurácia no conjunto de treinamento
train_accuracy = np.mean(train_predictions == y_train_sample)
# Calculando a acurácia no conjunto de teste
test_accuracy = np.mean(test_predictions == y_test_sample)

print(f"Acurácia no conjunto de treinamento: {train_accuracy*100:.2f}%")
print(f"Acurácia no conjunto de teste: {test_accuracy*100:.2f}%")


Acurácia no conjunto de treinamento: 99.81%
Acurácia no conjunto de teste: 99.83%


Dado o desequilíbrio claro no conjunto de dados, já que a maioria das transações são não fraudulentas, essa alta acurácia chega a ser duvidosa, pois é provável que o modelo esteja prevendo a classe majoritária (não fraude) para todas as entradas.

## Sobre Overfitting e Underfitting:

Primeiro vamos ver quais resultados são obtidos para diferentes valores de learning rate e épocas

In [17]:
# Definindo diferentes valores para learning rate e épocas
learning_rates = [0.1, 0.01, 0.001]
epochs_values = [10, 20, 50]

results = {}

# Treinando e avaliando o modelo para cada combinação de learning rate e épocas
for lr in learning_rates:
    for ep in epochs_values:
        # Instanciando e treinando o Perceptron
        perceptron = Perceptron(input_size=X_train_sample.shape[1])
        perceptron.train(X_train_sample, y_train_sample, lr, ep)
        
        # Avaliando o desempenho
        train_predictions = [perceptron.predict(x) for x in X_train_sample]
        test_predictions = [perceptron.predict(x) for x in X_test_sample]
        train_accuracy = np.mean(train_predictions == y_train_sample)
        test_accuracy = np.mean(test_predictions == y_test_sample)
        
        results[(lr, ep)] = (train_accuracy, test_accuracy)

results


{(0.1, 10): (0.998179209887442, 0.99833807333544),
 (0.1, 20): (0.998179209887442, 0.99833807333544),
 (0.1, 50): (0.9981541301338256, 0.9983263696265346),
 (0.01, 10): (0.9980688589715294, 0.9982678510820079),
 (0.01, 20): (0.9980688589715294, 0.9982561473731025),
 (0.01, 50): (0.9980839068236994, 0.9982561473731025),
 (0.001, 10): (0.9981039706265926, 0.9982912584998186),
 (0.001, 20): (0.9978632049918742, 0.998139110284049),
 (0.001, 50): (0.9980839068236994, 0.9982561473731025)}

In [23]:
df = pd.DataFrame(results).T.reset_index()
df.columns = ['Learning Rate', 'Épocas', 'Acurácia (Treinamento)', 'Acurácia (Teste)']

df

,Learning Rate,Épocas,Acurácia (Treinamento),Acurácia (Teste)
0,0.100,10,0.998179,0.998338
1,0.100,20,0.998179,0.998338
2,0.100,50,0.998154,0.998326
3,0.010,10,0.998069,0.998268
4,0.010,20,0.998069,0.998256
5,0.010,50,0.998084,0.998256
6,0.001,10,0.998104,0.998291
7,0.001,20,0.997863,0.998139
8,0.001,50,0.998084,0.998256


Chega a ser difícil caracterizar o nosso modelo, já que os resultados apresentarem alta acurácia, mas é bem provável que isso seja graças ao desbalanceamento do dataset.

Contudo, acredito que poderíamos avaliá-lo como um Overfitting, já que o modelo se ajustou muito bem ao conjunto de dados, tendo ótimos resultados tanto no conjunto de testes como no de treino.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2cc0a886-4130-43fc-9dc9-c8d9a5edaf78' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>